# Step 0a - Install dependencies

In [2]:
!pip install pandas numpy
!pip install torch torchvision torchaudio
!pip install datasets sentencepiece



  Using cached datasets-2.18.0-py3-none-any.whl.metadata (20 kB)
  Using cached sentencepiece-0.2.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.7 kB)
  Using cached pyarrow-15.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.0 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.4.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.2.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached aiohttp-3.9.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.4 kB)
  Using cached PyYAML-6.0.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  

# Step 0b - Import module dependencies

In [14]:
import os
import random
from datasets import load_dataset
import sentencepiece as spm

# Step 0c - Constants

In [12]:
sentencepiece_output_dir = 'sentencepiece_models'
sentencepiece_corpus_filename = f"tiny_stories_texts.txt"

# Step 0c - Load datasets

Read the tiny stories data set:

In [5]:
# Load the Tiny Stories dataset
dataset = load_dataset("roneneldan/TinyStories")

# Split the dataset into training and validation sets
train_dataset = dataset['train']
valid_dataset = dataset['validation']

Repo card metadata block was not found. Setting CardData to empty.
Generating train split: 2119719 examples [00:02, 717132.82 examples/s]
Generating validation split: 21990 examples [00:00, 850651.55 examples/s]


Gather all of the data set and export it to a text file for training of the sentence piece model:

In [15]:
# Specify the directory where you want to save the files
if not os.path.exists(sentencepiece_output_dir):
    os.makedirs(sentencepiece_output_dir)

# Save all texts to a single file in the specified directory, one story per line
sentencepiece_corpus_file_path = os.path.join(sentencepiece_output_dir, sentencepiece_corpus_filename)


# Combine texts from training and validation sets
all_texts = train_dataset['text'] + valid_dataset['text']

random.shuffle(all_texts)

# Sample a smaller subset of the dataset, e.g., 10% of the data
sample_size = int(0.1 * len(all_texts))
sampled_text = all_texts[:sample_size]

# Save all texts to a single file, one story per line
with open(sentencepiece_corpus_file_path, 'w', encoding='utf-8') as f:
    for story in sampled_text:
        f.write(story + '\n')

Next generate the sentence piece model:

In [16]:
spm.SentencePieceTrainer.train(input=sentencepiece_corpus_file_path, model_prefix=os.path.join(sentencepiece_output_dir, 'tiny_stories_spm_sampled'), vocab_size=8000, character_coverage=0.9995, model_type='unigram')

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: sentencepiece_models/tiny_stories_texts.txt
  input_format: 
  model_prefix: sentencepiece_models/tiny_stories_spm_sampled
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piec